In [41]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/digit-recognizer/sample_submission.csv
/kaggle/input/digit-recognizer/train.csv
/kaggle/input/digit-recognizer/test.csv


In [42]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

data = pd.read_csv("/kaggle/input/digit-recognizer/train.csv")

In [43]:
data.head()

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [44]:
data.shape

(42000, 785)

In [45]:
data = np.array(data)
m , n = data.shape

np.random.shuffle(data)

data_dev  = data[0:1000].T

Y_dev = data_dev[0]
X_dev = data_dev[1: n] /255

data_train = data[1000: m].T

Y_train = data_train[0]
X_train = data_train[1:n] /255

In [46]:
print(X_train.shape)
print(Y_train.shape)
print(X_dev.shape)
print(Y_dev.shape)

(784, 41000)
(41000,)
(784, 1000)
(1000,)


Applying a 2 Layer Neural Network on the MNIST Dataset

In [47]:
def init_params():
    W1 = np.random.rand(10, 784) * 0.01
    b1 = np.zeros((10, 1)) 
    W2 = np.random.rand(10, 10) * 0.01
    b2 = np.zeros((10, 1)) 
    return W1, b1, W2, b2

def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    expZ = np.exp(Z - np.max(Z, axis=0, keepdims=True))  
    return expZ / np.sum(expZ, axis=0, keepdims=True)

    
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y):
    m = X.shape[1]
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2, axis = 1, keepdims = True)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1, axis = 1, keepdims = True)
    return dW1, db1, dW2, db2

def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

In [48]:
def get_predictions(A2):
    return np.argmax(A2, axis=0)

def get_accuracy(predictions, Y):
    return np.mean(predictions == Y)

def compute_cost(A2, Y):
    m = Y.shape[0]
    one_hot_Y = one_hot(Y)
    cost = -np.sum(one_hot_Y * np.log(A2 + 1e-8)) / m
    return np.squeeze(cost)

def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        
        if i % 10 == 0:
            predictions = get_predictions(A2)
            acc = get_accuracy(predictions, Y)
            cost = compute_cost(A2, Y)
            print(f"Iteration {i}: Accuracy = {acc:.4f}, Cost = {cost:.4f}")
    
    return W1, b1, W2, b2


In [49]:
gradient_descent(X_train,Y_train,0.1, 250)

Iteration 0: Accuracy = 0.0973, Cost = 2.3029
Iteration 10: Accuracy = 0.0986, Cost = 2.2953
Iteration 20: Accuracy = 0.0986, Cost = 2.2854
Iteration 30: Accuracy = 0.1089, Cost = 2.2679
Iteration 40: Accuracy = 0.1823, Cost = 2.2361
Iteration 50: Accuracy = 0.2045, Cost = 2.1950
Iteration 60: Accuracy = 0.2155, Cost = 2.1519
Iteration 70: Accuracy = 0.2273, Cost = 2.1109
Iteration 80: Accuracy = 0.2498, Cost = 2.0708
Iteration 90: Accuracy = 0.2983, Cost = 2.0263
Iteration 100: Accuracy = 0.3603, Cost = 1.9684
Iteration 110: Accuracy = 0.4207, Cost = 1.8860
Iteration 120: Accuracy = 0.4729, Cost = 1.7723
Iteration 130: Accuracy = 0.5265, Cost = 1.6391
Iteration 140: Accuracy = 0.5768, Cost = 1.5106
Iteration 150: Accuracy = 0.6184, Cost = 1.3983
Iteration 160: Accuracy = 0.6518, Cost = 1.3016
Iteration 170: Accuracy = 0.6822, Cost = 1.2171
Iteration 180: Accuracy = 0.7060, Cost = 1.1417
Iteration 190: Accuracy = 0.7295, Cost = 1.0728
Iteration 200: Accuracy = 0.7505, Cost = 1.0086
Ite

(array([[0.00876279, 0.00978084, 0.00953262, ..., 0.00062721, 0.00099877,
         0.00098969],
        [0.00991222, 0.00424956, 0.0063887 , ..., 0.00811952, 0.00199422,
         0.0001794 ],
        [0.00265209, 0.0042673 , 0.00039921, ..., 0.00848567, 0.00475676,
         0.00224299],
        ...,
        [0.00956427, 0.00031179, 0.00552014, ..., 0.00794474, 0.00509521,
         0.00227755],
        [0.00292668, 0.00483456, 0.00936674, ..., 0.00961546, 0.00568319,
         0.0029449 ],
        [0.0097615 , 0.00228501, 0.00809776, ..., 0.0069181 , 0.00635573,
         0.00497031]]),
 array([[ 0.01349249],
        [ 0.01721473],
        [-0.0186533 ],
        [ 0.02991078],
        [-0.05497286],
        [ 0.09436705],
        [ 0.02446769],
        [ 0.03951623],
        [-0.00313176],
        [-0.01007865]]),
 array([[ 0.03858166,  0.78108326,  0.0898303 , -0.02728367,  0.12051675,
         -0.16460699,  0.54112784,  0.55229294, -0.20183595,  0.02699992],
        [-0.13489168, -0.189